## Show a warning for removing all the project threads

In [1]:
# raise Exception(f"You will remove all your project thread permanently.\nComment this line to proceed, if you know what you are doing!")

In [14]:
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv(dotenv_path=".env", override=True)

from utils.fdyauth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


In [15]:
from azure.ai.projects import AIProjectClient
from azure.core.exceptions import ResourceNotFoundError
import azure.ai.agents as agentslib
import azure.ai.projects as projectlib

project_client = AIProjectClient(
    credential=credential,
    endpoint=settings.project_endpoint,
    api_version=settings.project_api_version
)
print("project_client api version:", project_client._config.api_version)
print(f"azure-ai-agents version: {agentslib.__version__}")
print(f"azure-ai-projects version: {projectlib.__version__}")

project_client api version: 2025-05-15-preview
azure-ai-agents version: 1.1.0b4
azure-ai-projects version: 1.0.0b12


In [16]:
threads = project_client.agents.threads.list()
threads_count = 0
for thread in threads:
    threads_count += 1
    # print(f"Thread ID: {thread.id}, Created At: {thread.created_at}")
print(f"total threads: {threads_count}")

total threads: 0


### ResourceNotFoundError

Note:
* If the ResourceNotFoundError occurs, the notebook thread will be breaken, since there is an async raise in the sdk. You can not catch it.
* Just rerun the code to clean the threads. The not available thread will be removed from list anyway.

In [ ]:
# Collect thread IDs first to avoid iteration issues with lazy loading
thread_ids = []
try:
    threads = project_client.agents.threads.list()
    for thread in threads:
        thread_ids.append(thread.id)
except ResourceNotFoundError as e:
    print(f"Warning: Some threads may no longer exist: {e}")

deleted_threads_count = 0
not_found_count = 0
not_found_threads = []

print(f"Found {len(thread_ids)} threads to process")

# Now delete each thread by ID
for thread_id in thread_ids:
    try:
        # sync call
        project_client.agents.threads.delete(thread_id)
        deleted_threads_count += 1
        print(f"Deleted thread: {thread_id}")
    except ResourceNotFoundError:
        not_found_count += 1
        not_found_threads.append(thread_id)
        print(f"Thread not found (already deleted?): {thread_id}")
    except Exception as e:
        print(f"Failed to delete thread {thread_id}: {e}")

print(f"\nSummary:")
print(f"Deleted threads: {deleted_threads_count}")
print(f"Threads not found: {not_found_count}")

Found 0 threads to process

Summary:
Deleted threads: 0
Threads not found: 0
